In [1]:
# shutil.copyfile("/content/END_Assignments/Capstone/english_python_data.txt", "/content/END_Assignments/Capstone/python_data_100.txt")

In [2]:
# https://github.com/abishek-raju/END_Assignments.git

In [3]:
import shutil
try:
  shutil.rmtree("./END_Assignments")
except:
  pass

In [4]:
!git clone -b assignment https://github.com/abishek-raju/END_Assignments.git


Cloning into 'END_Assignments'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 12 (delta 2), reused 7 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.


In [5]:
!cd "./END_Assignments/Capstone"

In [6]:
import os
os.chdir('./END_Assignments/Capstone')

In [7]:
os.listdir()

['english_python_data.txt']

In [8]:
f = open('english_python_data.txt', 'r')
full_text = f.read()              # Read entire file into a string

f.close()

In [9]:
# full_text[0:150]

In [10]:
# len(full_text)

In [11]:
# f = open('/content/Untitled (4).txt', 'r')
# full_text = f.read()              # Read entire file into a string

# f.close()
# x = full_text.split("\n# ")
# len(x)

In [12]:
x = full_text.split("\n#")

In [13]:
len(x)

4958

In [14]:
x[0]

"# write a python program to add two numbers \nnum1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n"

In [15]:
queue_raw,snippet_raw = [],[]
for count,i in enumerate(x):
  q_a = i.split("\n",1)
  if len(q_a)>1:
    if q_a[1]:
      # print("code example : "+str(count)+"\n")
      # print(q_a[0])
      queue_raw.append(q_a[0])

      # print("\n")
      # print(q_a[1])
      snippet_raw.append(q_a[1])
      # print("-*-"*10)

In [16]:
snippet_raw[1]

'def add_two_numbers(num1, num2):\n    sum = num1 + num2\n    return sum\n\n'

In [17]:
bool(x[100].split("\n")[1])

False

In [18]:
x[0]

"# write a python program to add two numbers \nnum1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n"

In [19]:
# for count,i in enumerate(x):
#   q_a = i.split("\n",1)
#   if len(q_a)>1:
#     print("code example : "+str(count)+"\n")
#     print(q_a[0])

#     print("\n")
#     print(q_a[1])
#     print("-*-"*10)

In [20]:
# ToDo
# 1.Preprocess
#     -remove white space at the start of the snippet
#     -remove numbers at the start of the snippet

In [21]:
import torch
from torchtext.legacy import data 

In [22]:
tokens_to_discard = ["\n"," ","  "]


In [23]:
# [i for i in [" ","hi","i"] if i not in tokens_to_discard]

In [24]:
def code_tokenize(text):
  token_stream = tokenize.generate_tokens(StringIO(text).readline)
  code_tokenized = [token.string for token in token_stream]
  if code_tokenized[0] in tokens_to_discard:
    return code_tokenized[1:]
  else:
    return code_tokenized
  
  # temp = []
  # for token in token_stream:
  #   temp.append(token.string)
  #   print(token)

In [25]:
queue = data.Field(sequential = True, tokenize = 'spacy',eos_token = '<eos>', init_token = '<sos>',  batch_first =True, include_lengths=True)
snippet = data.Field(sequential = True,is_target=True ,eos_token = '<eos>', init_token = '<sos>', tokenize = code_tokenize, batch_first =True, include_lengths=True)

In [26]:
fields = [('queue', queue),('snippet',snippet)]

In [27]:
# queue_raw,snippet_raw

In [28]:
import tokenize
try:
  from StringIO import StringIO
except:
  from io import StringIO

In [29]:
# example = [data.Example.fromlist([queue_raw[i],snippet_raw[i]], fields) for i in range(len(queue_raw))] 
example = []
for i in range(len(queue_raw)):
  try:
    temp = data.Example.fromlist([queue_raw[i],snippet_raw[i]], fields)
    if len(vars(temp)["snippet"]) < 500:
      example.append(temp)
    else:
      pass
  except:
    print(i)

463
465
643
686
687
814
1056
1060
1418
1808
1935
3441
3481
3647
3673
3840
3853
3875
3876
3877
4059
4063


In [30]:
max([len(vars(i)["snippet"]) for i in example])
# 785

438

In [31]:
# temp = [len(vars(i)["snippet"]) for i in example]

In [32]:
# temp.sort(reverse=True)

In [33]:
# temp

In [34]:
queue_raw[1],snippet_raw[0]

(' write a python function to add two user provided numbers and return the sum',
 "num1 = 1.5\nnum2 = 6.3\nsum = num1 + num2\nprint(f'Sum: {sum}')\n\n")

In [35]:
len(queue_raw)

4741

In [36]:
queue_snippet_dataset = data.Dataset(example, fields)

In [37]:
(train_data, valid_data) = queue_snippet_dataset.split(split_ratio=[0.85, 0.15])

In [38]:
len(train_data),len(valid_data)

(4010, 708)

In [39]:
queue.build_vocab(queue_snippet_dataset)
snippet.build_vocab(queue_snippet_dataset)


In [40]:
print('Size of input vocab : ', len(queue.vocab))
print('Size of label vocab : ', len(snippet.vocab))
print('Top 10 words appreared repeatedly :', list(queue.vocab.freqs.most_common(10)))
print('Labels : ', snippet.vocab.stoi)

Size of input vocab :  2806
Size of label vocab :  6861
Top 10 words appreared repeatedly : [('a', 5976), (' ', 4217), ('to', 3565), ('the', 2446), ('of', 2065), ('write', 1984), ('function', 1834), ('python', 1783), ('program', 1691), ('Write', 1527)]
Labels :  defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x7f46a435e4d0>>, {'<unk>': 0, '<pad>': 1, '<sos>': 2, '<eos>': 3, '\n': 4, '(': 5, ')': 6, '': 7, ',': 8, ':': 9, '=': 10, '[': 11, ']': 12, 'print': 13, '.': 14, '1': 15, 'i': 16, '    ': 17, 'def': 18, 'in': 19, 'return': 20, '0': 21, 'for': 22, '        ': 23, 'if': 24, '+': 25, '2': 26, 'n': 27, 'x': 28, '-': 29, '==': 30, 'a': 31, '*': 32, '3': 33, 'range': 34, 'len': 35, 'num': 36, '5': 37, 'else': 38, 'j': 39, '4': 40, 'import': 41, '            ': 42, 'res': 43, 'self': 44, 'str': 45, '{': 46, '}': 47, '%': 48, 'b': 49, 's': 50, '10': 51, 'str1': 52, '6': 53, 'y': 54, 'float': 55, 'list': 56, '+=': 57, 'key': 58, 'int': 59, 'count': 

In [41]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [42]:
train_iterator, valid_iterator = data.BucketIterator.splits((train_data, valid_data), batch_size = 32, 
                                                            sort_key = lambda z: len(z.queue),
                                                            sort_within_batch=True, device = device)

In [43]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(snippet.vocab.stoi, tokens)

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext
from torchtext.legacy.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy
import numpy as np

import random
import math
import time

In [45]:
class Encoder(nn.Module):
    def __init__(self, 
                 input_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim,
                 dropout, 
                 device,
                 max_length = 100):
        super().__init__()

        self.device = device
        
        self.tok_embedding = nn.Embedding(input_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([EncoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim,
                                                  dropout, 
                                                  device) 
                                     for _ in range(n_layers)])
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len]
        #src_mask = [batch size, 1, 1, src len]
        
        batch_size = src.shape[0]
        src_len = src.shape[1]
        
        pos = torch.arange(0, src_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
        
        #pos = [batch size, src len]
        
        src = self.dropout((self.tok_embedding(src) * self.scale) + self.pos_embedding(pos))
        
        #src = [batch size, src len, hid dim]
        
        for layer in self.layers:
            src = layer(src, src_mask)
            
        #src = [batch size, src len, hid dim]
            
        return src

In [46]:
class EncoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim,  
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, src_mask):
        
        #src = [batch size, src len, hid dim]
        #src_mask = [batch size, 1, 1, src len] 
                
        #self attention
        _src, _ = self.self_attention(src, src, src, src_mask)
        
        #dropout, residual connection and layer norm
        src = self.self_attn_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        #positionwise feedforward
        _src = self.positionwise_feedforward(src)
        
        #dropout, residual and layer norm
        src = self.ff_layer_norm(src + self.dropout(_src))
        
        #src = [batch size, src len, hid dim]
        
        return src

In [47]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        
        assert hid_dim % n_heads == 0
        
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        
        self.fc_o = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, query, key, value, mask = None):
        
        batch_size = query.shape[0]
        
        #query = [batch size, query len, hid dim]
        #key = [batch size, key len, hid dim]
        #value = [batch size, value len, hid dim]
                
        Q = self.fc_q(query)
        K = self.fc_k(key)
        V = self.fc_v(value)
        
        #Q = [batch size, query len, hid dim]
        #K = [batch size, key len, hid dim]
        #V = [batch size, value len, hid dim]
                
        Q = Q.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(batch_size, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        
        #Q = [batch size, n heads, query len, head dim]
        #K = [batch size, n heads, key len, head dim]
        #V = [batch size, n heads, value len, head dim]
                
        energy = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        
        #energy = [batch size, n heads, query len, key len]
        
        if mask is not None:
            energy = energy.masked_fill(mask == 0, -1e10)
        
        attention = torch.softmax(energy, dim = -1)
                
        #attention = [batch size, n heads, query len, key len]
                
        x = torch.matmul(self.dropout(attention), V)
        
        #x = [batch size, n heads, query len, head dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        
        #x = [batch size, query len, n heads, head dim]
        
        x = x.view(batch_size, -1, self.hid_dim)
        
        #x = [batch size, query len, hid dim]
        
        x = self.fc_o(x)
        
        #x = [batch size, query len, hid dim]
        
        return x, attention

In [48]:
class PositionwiseFeedforwardLayer(nn.Module):
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        
        self.fc_1 = nn.Linear(hid_dim, pf_dim)
        self.fc_2 = nn.Linear(pf_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        
        #x = [batch size, seq len, hid dim]
        
        x = self.dropout(torch.relu(self.fc_1(x)))
        
        #x = [batch size, seq len, pf dim]
        
        x = self.fc_2(x)
        
        #x = [batch size, seq len, hid dim]
        
        return x

In [49]:
class Decoder(nn.Module):
    def __init__(self, 
                 output_dim, 
                 hid_dim, 
                 n_layers, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device,
                 max_length = 450):
        super().__init__()
        
        self.device = device
        
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(self.device)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)
                            
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        # print("deeeeeeeeeeeeeeeeeeeeeeeeeeecoder here")
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, enc_src, trg_mask, src_mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        
        #output = [batch size, trg len, output dim]
            
        return output, attention

In [50]:
class DecoderLayer(nn.Module):
    def __init__(self, 
                 hid_dim, 
                 n_heads, 
                 pf_dim, 
                 dropout, 
                 device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.enc_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.encoder_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, 
                                                                     pf_dim, 
                                                                     dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, enc_src, trg_mask, src_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, _ = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
            
        #encoder attention
        _trg, attention = self.encoder_attention(trg, enc_src, enc_src, src_mask)
        # query, key, value
        
        #dropout, residual connection and layer norm
        trg = self.enc_attn_layer_norm(trg + self.dropout(_trg))
                    
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return trg, attention

In [51]:
class Seq2Seq(nn.Module):
    def __init__(self, 
                 encoder, 
                 decoder, 
                 src_pad_idx, 
                 trg_pad_idx, 
                 device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.src_pad_idx = src_pad_idx
        self.trg_pad_idx = trg_pad_idx
        self.device = device
        
    def make_src_mask(self, src):
        
        #src = [batch size, src len]
        
        src_mask = (src != self.src_pad_idx).unsqueeze(1).unsqueeze(2)

        #src_mask = [batch size, 1, 1, src len]

        return src_mask
    
    def make_trg_mask(self, trg):
        
        #trg = [batch size, trg len]
        
        trg_pad_mask = (trg != self.trg_pad_idx).unsqueeze(1).unsqueeze(2)
        
        #trg_pad_mask = [batch size, 1, 1, trg len]
        
        trg_len = trg.shape[1]
        
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device = self.device)).bool()
        
        #trg_sub_mask = [trg len, trg len]
            
        trg_mask = trg_pad_mask & trg_sub_mask
        
        #trg_mask = [batch size, 1, trg len, trg len]
        
        return trg_mask

    def forward(self, src, trg):
        
        #src = [batch size, src len]
        #trg = [batch size, trg len]
                
        src_mask = self.make_src_mask(src)
        trg_mask = self.make_trg_mask(trg)
        
        #src_mask = [batch size, 1, 1, src len]
        #trg_mask = [batch size, 1, trg len, trg len]
        # print("mask_generated")
        enc_src = self.encoder(src, src_mask)
        # print("encoded_info")
        #enc_src = [batch size, src len, hid dim]
                
        output, attention = self.decoder(trg, enc_src, trg_mask, src_mask)
        # print("decoded_info")
        #output = [batch size, trg len, output dim]
        #attention = [batch size, n heads, trg len, src len]
        
        return output, attention

In [52]:
INPUT_DIM = len(queue.vocab)
OUTPUT_DIM = len(snippet.vocab)
HID_DIM = 128
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)

dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT, 
              device)

In [53]:
QUEUE_PAD_IDX = queue.vocab.stoi[queue.pad_token]
SNIPPET_PAD_IDX = snippet.vocab.stoi[snippet.pad_token]

model = Seq2Seq(enc, dec, QUEUE_PAD_IDX, SNIPPET_PAD_IDX, device).to(device)

In [54]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 3,581,389 trainable parameters


In [55]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [56]:
model.apply(initialize_weights);

In [57]:
LEARNING_RATE = 0.0005

optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)

In [58]:
criterion = nn.CrossEntropyLoss(ignore_index = SNIPPET_PAD_IDX)


In [59]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src,_ = batch.queue
        trg,_ = batch.snippet
        optimizer.zero_grad()
        # print(src.shape,trg.shape)
        # print(trg)
        output, _ = model(src, trg[:,:-1])
        #output = [batch size, trg len - 1, output dim]
        #trg = [batch size, trg len]
            
        output_dim = output.shape[-1]
            
        output = output.contiguous().view(-1, output_dim)
        trg = trg[:,1:].contiguous().view(-1)
                
        #output = [batch size * trg len - 1, output dim]
        #trg = [batch size * trg len - 1]
            
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [60]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):
            src,_ = batch.queue
            trg,_ = batch.snippet

            output, _ = model(src, trg[:,:-1])
            
            #output = [batch size, trg len - 1, output dim]
            #trg = [batch size, trg len]
            
            output_dim = output.shape[-1]
            
            output = output.contiguous().view(-1, output_dim)
            trg = trg[:,1:].contiguous().view(-1)
            
            #output = [batch size * trg len - 1, output dim]
            #trg = [batch size * trg len - 1]
            
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [61]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [62]:
N_EPOCHS = 50
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 20s
	Train Loss: 5.236 | Train PPL: 187.890
	 Val. Loss: 3.557 |  Val. PPL:  35.047
Epoch: 02 | Time: 0m 20s
	Train Loss: 3.204 | Train PPL:  24.633
	 Val. Loss: 2.873 |  Val. PPL:  17.688
Epoch: 03 | Time: 0m 20s
	Train Loss: 2.747 | Train PPL:  15.592
	 Val. Loss: 2.588 |  Val. PPL:  13.304
Epoch: 04 | Time: 0m 20s
	Train Loss: 2.483 | Train PPL:  11.979
	 Val. Loss: 2.405 |  Val. PPL:  11.081
Epoch: 05 | Time: 0m 20s
	Train Loss: 2.286 | Train PPL:   9.836
	 Val. Loss: 2.283 |  Val. PPL:   9.808
Epoch: 06 | Time: 0m 20s
	Train Loss: 2.118 | Train PPL:   8.314
	 Val. Loss: 2.155 |  Val. PPL:   8.627
Epoch: 07 | Time: 0m 20s
	Train Loss: 1.973 | Train PPL:   7.195
	 Val. Loss: 2.073 |  Val. PPL:   7.949
Epoch: 08 | Time: 0m 20s
	Train Loss: 1.835 | Train PPL:   6.263
	 Val. Loss: 1.973 |  Val. PPL:   7.193
Epoch: 09 | Time: 0m 20s
	Train Loss: 1.720 | Train PPL:   5.583
	 Val. Loss: 1.899 |  Val. PPL:   6.680
Epoch: 10 | Time: 0m 20s
	Train Loss: 1.610 | Train PPL

In [63]:
# model.load_state_dict(torch.load('tut6-model.pt'))

# test_loss = evaluate(model, valid_iterator, criterion)

# print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

In [64]:
def translate_sentence(sentence, src_field, trg_field, model, device, max_len = 50):
    
    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    
    src_mask = model.make_src_mask(src_tensor)
    
    with torch.no_grad():
        enc_src = model.encoder(src_tensor, src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        trg_mask = model.make_trg_mask(trg_tensor)
        
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:], attention

In [80]:
example_idx = 21

src = vars(train_data.examples[example_idx])['queue']
trg = vars(train_data.examples[example_idx])['snippet']

print(f'src = {src}')
print(f'trg = {trg}')

src = [' ', 'write', 'a', 'python', 'program', 'to', 'print', 'bitwise', 'AND', 'operation']
trg = ['a', '=', '60', '\n', 'b', '=', '13', '\n', 'a_and_b', '=', 'a', '&', 'b', '\n', 'print', '(', 'a_and_b', ')', '', '']


In [105]:
src,trg = vars(example[1])['queue'],vars(example[1])['snippet']

print("queue:")
print(" ".join(src))
print("*"*50)
print("\nsnippet:")
print(" ".join(trg))
print("*"*50)
translation, attention = translate_sentence(src, queue, snippet, model, device)
# print(f'predicted trg = {translation}')
print("\npredicted:")
print(" ".join(translation[:-1]))

queue:
  write a python function to add two user provided numbers and return the sum
**************************************************

snippet:
def add_two_numbers ( num1 , num2 ) : 
      sum = num1 + num2 
 return sum   
**************************************************

predicted:
def add_two_numbers ( num1 , num2 ) : 
      sum = num1 + num2 
 return sum   


In [112]:
for i in range(10):
  src,trg = vars(example[i])['queue'],vars(example[i])['snippet']
  print("#"*100)
  print("queue:")
  print(" ".join(src))
  print("*"*50)
  print("\nsnippet:")
  print(" ".join(trg))
  print("*"*50)
  translation, attention = translate_sentence(src, queue, snippet, model, device)
  # print(f'predicted trg = {translation}')
  print("\npredicted:")
  print(" ".join(translation[:-1]))
  print("#"*100)
  print("")

####################################################################################################
queue:
# write a python program to add two numbers
**************************************************

snippet:
num1 = 1.5 
 num2 = 6.3 
 sum = num1 + num2 
 print ( f'Sum: {sum}' )  
**************************************************

predicted:
num1 = 1.5 
 num2 = 6.3 
 sum = num1 + num2 
 print ( f'Sum: {sum}' )  
####################################################################################################

####################################################################################################
queue:
  write a python function to add two user provided numbers and return the sum
**************************************************

snippet:
def add_two_numbers ( num1 , num2 ) : 
      sum = num1 + num2 
 return sum   
**************************************************

predicted:
def add_two_numbers ( num1 , num2 ) : 
      sum = num1 + num2 
 return sum   
##########

In [113]:
# src = ["add","two","numbers"]
# src = ["Convert", "a", "string", "to", "an", "integer"]
# src = ["Get", "a", "list", "of", "values", "from", "a", "list", "of", "dictionaries", "in", "python"]
src = ["write","a","function","that","generates","fibbonacci","series"]

In [114]:
translation, attention = translate_sentence(src, queue, snippet, model, device)

print(f'predicted trg = {translation}')

predicted trg = ['def', 'fib', '(', 'n', ')', ':', '\n', '   ', 'if', 'n', '==', '1', ':', '\n', '      ', 'return', 'n', '\n', '', 'else', ':', '\n', '      ', 'return', 'n', '*', 'fib', '(', 'n', '-', '1', ')', '+', 'fib', '(', 'n', '-', '2', ')', '', '', '', '', '<eos>']


In [68]:
from torchtext.data.metrics import bleu_score

def calculate_bleu(data, src_field, trg_field, model, device, max_len = 50):
    
    trgs = []
    pred_trgs = []
    
    for datum in data:
        
        src = vars(datum)['queue']
        trg = vars(datum)['snippet']
        
        pred_trg, _ = translate_sentence(src, src_field, trg_field, model, device, max_len)
        
        #cut off <eos> token
        pred_trg = pred_trg[:-1]
        
        pred_trgs.append(pred_trg)
        trgs.append([trg])
        
    return bleu_score(pred_trgs, trgs)

In [69]:
# bleu_score = calculate_bleu(valid_data, queue, snippet, model, device)

# print(f'BLEU score = {bleu_score*100:.2f}')